In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread
import numpy as np
import pandas as pd
import cv2
from skimage.util import montage
from skimage import io
import shutil

%matplotlib inline
tf.__version__

import sys
print(sys.version)
print(sys.executable)

import math
from PIL import Image, ImageDraw, ImageColor
import json
import glob
import os
import cv2
import nibabel as nib

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\Anaconda3\python.exe


In [13]:
# draw dicom images with metadata info
from datetime import datetime
def drawCategorizedImages(folderPath, dict, fontsize=10, fontfamily='MingLiU'):
    for patient in dict:
        for study in dict[patient]:
            for series in dict[patient][study]:
                fileList = dict[patient][study][series]
                imagesList = []
                for file in fileList:
                    imagesList.append(dcmread(folderPath + file['dcmPath']))
                imagesList.sort(key=lambda x: x.InstanceNumber, reverse=False) # sort image by instance number
                quantity = len(imagesList)
                cols = 10
                rows = math.ceil(quantity/cols)
                rows = rows + int(rows/6) + 1 # add blank rows to place text
                fig = plt.figure(figsize=(cols*2,rows*2))
                flag = True
                for i in range(quantity):
                    ds = imagesList[i]

                    if ("PixelData" in ds):
                        #add text only once
                        if flag:
                            plt.text(0.01,0.14,"PatientName: " + str(ds.PatientName), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.01,0.11,"PatientID: " + str(ds.PatientID), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.01,0.08,"Sex: " + str(ds.PatientSex), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.14,"InstitutionName: " + ds.InstitutionName if ds.InstitutionName != "" else "InstitutionName: None", fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.11,"StudyDescription: " + ds.StudyDescription, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.33,0.08,"StudyID: " + ds.StudyID, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.14,"SeriesNumber: " + str(ds.SeriesNumber), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.11,"SeriesDescription: " + str(ds.SeriesDescription), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.08,"AccessionNumber: " + ds.AccessionNumber, fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.05,"AcquisitionDate: " + str(datetime.strptime(ds.AcquisitionDate, '%Y%m%d').date()), fontsize=fontsize, fontfamily=fontfamily)
                            plt.text(0.66,0.02,"AcquisitionTime: " + ds.AcquisitionTime, fontsize=fontsize, fontfamily=fontfamily)
#                             plt.text(0.66,0.085,"SliceThickness: " + str(math.ceil(ds.SliceThickness)) + "mm", fontsize=fontsize, fontfamily=fontfamily)
#                             plt.text(0.66,0.06,"SliceLocation: " + str(math.ceil(ds.SliceLocation)) + "mm", fontsize=fontsize, fontfamily=fontfamily)
                            flag = False   

                        fig.add_subplot(rows, cols, i+1)
                        if(len(ds.pixel_array.shape)==3):
                            plt.imshow(ds.pixel_array[:,:,1])
                        else:
                            plt.imshow(ds.pixel_array)
                        plt.axis('off')

##################################################################

def showImagesWithPyPlot(images,cmap):
    quantity = len(images)
    cols = 9
    rows = math.ceil(quantity/cols)
    fig,ax = plt.subplots(rows,cols,figsize=(22,2.5*rows))
    for i in range(quantity):
        ax[i//cols][i%cols].imshow(images[i],cmap=cmap)
        ax[i//cols][i%cols].axis('off')

def showNiftiImagesWithPyPlot(nifti,cmap):
    quantity = nifti.shape[2]
    cols = 9
    rows = math.ceil(quantity/cols)
    fig,ax = plt.subplots(rows,cols,figsize=(22,2.5*rows))
    for i in range(quantity):
        ax[i//cols][i%cols].imshow(nifti[:,:,i],cmap=cmap)
        ax[i//cols][i%cols].axis('off')

##################################################################

#使用json檔案畫實心標記區域
def fillMissingPoint(segments):
    startX = segments[0]['a'][0]
    startY = segments[0]['a'][1]
    endX = segments[len(segments)-1]['b'][0]
    endY = segments[len(segments)-1]['b'][1]
    width = segments[0]['width']
    distanceX = abs(endX-startX)
    distanceY = abs(endY-startY)    
    stepX = 1
    if(distanceX<=distanceY):
        stepX = distanceX/distanceY
    stepY = 1
    if(distanceY<=distanceX):
        stepY = distanceY/distanceX
    if(endX >= startX and endY >= startY):
        while(endX >= startX and endY >= startY):
            a = [endX,endY]
            endX-=stepX
            endY-=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX >= startX and endY <= startY):
        while(endX >= startX and endY <= startY):
            a = [endX,endY]
            endX-=stepX
            endY+=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX <= startX and endY <= startY):
        while(endX <= startX and endY <= startY):
            a = [endX,endY]
            endX+=stepX
            endY+=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    elif(endX <= startX and endY >= startY):
        while(endX <= startX and endY >= startY):
            a = [endX,endY]
            endX+=stepX
            endY-=stepY
            b = [endX,endY]
            segments.append({'a':a,'b':b,'width':width})
    return segments

def getLabeledImageWithJson(jsonPath):
    file = open(jsonPath)
    data = json.load(file)
    image = Image.new("RGB", data['size'])
    draw = ImageDraw.Draw(image)
    shapes = data['shapes']
    for shape in shapes:
        segments = shape['segments']
        segments = fillMissingPoint(segments)
        strokeColor = shape['strokeColor']
        for segment in segments:
            startEndCoord = [(segment['a'][0], segment['a'][1]),(segment['b'][0], segment['b'][1])]
            draw.line(startEndCoord, fill=strokeColor)
    
    grayImage  = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(grayImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(grayImage,[c], 0, (255,255,255), -1)

    colorImage = cv2.cvtColor(grayImage,cv2.COLOR_GRAY2RGB)
    colorImage[np.all(colorImage == (255, 255, 255), axis=-1)] = ImageColor.getcolor(strokeColor, "RGB")    
    
    #convert black background to transparent
    _, alpha = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY)
    b, g, r = cv2.split(colorImage)
    rgba = [b, g, r, alpha]
    colorImage = cv2.merge(rgba, 4)
    
    #convert cv2 to pil format
    pilImage = Image.fromarray(colorImage)
    return pilImage

##################################################################

# categorize dicom files with patientID, studyID and seriesID without using dicomdir
# 查找自己的json標記檔是否存在，如果不存在label設爲no，否則label設爲json標記檔的路徑
def getLabelPath(dcmPath, sopInstanceUID):
    jsonFiles = glob.glob(dcmPath.split('\\')[0] + "/*.json")
    for jsonFile in jsonFiles:
        if(sopInstanceUID in jsonFile):
            return jsonFile
    return 'NA'

# dict: {patientID->studyID->seriesID->{dcmPath,labelPath}}
def getCategorizedDictByFolder(path):
    files = glob.glob(path + "*.dcm")
    dict = {}
    for file in files:
        ds = dcmread(file)
        patientID = ds.PatientID
        studyID = ds.StudyID
        seriesID = str(ds.SeriesNumber)
        labelPath = getLabelPath(file, ds.SOPInstanceUID)
        data = {'dcmPath':file,'labelPath':labelPath}
        if patientID in dict:
            if studyID in dict[patientID]:
                if seriesID in dict[patientID][studyID]:
                    dict[patientID][studyID][seriesID].append(data)
                else:
                    dict[patientID][studyID][seriesID] = [data]
            else:
                dict[patientID][studyID] = {seriesID:[data]}
        else:
            dict[patientID] = {studyID:{seriesID:[data]}}

    # sort dict by InstanceNumber
    for patient in dict:
            for study in dict[patient]:
                for series in dict[patient][study]:
                    dict[patient][study][series].sort(key=lambda x: dcmread(x['dcmPath']).InstanceNumber, reverse=False)

    return dict

def getSeriesIDByName(dict, name):
    for patient in dict:
        for study in dict[patient]:
            for series in dict[patient][study]:
                for i in dict[patient][study][series]:
                    if(name.lower() in dcmread(i['dcmPath']).SeriesDescription.lower()):
                        return series
##################################################################

# 調整圖像色階
def adjust_color_scale(img,Shadow,Highlight):
    if Highlight > 255:
        Highlight = 255
    if Shadow < 0:
        Shadow = 0
    if Shadow >= Highlight:
        Shadow = Highlight - 2
    # 转类型
    img = np.array(img, dtype=int)
    # 计算白场黑场离差
    Diff = Highlight - Shadow
    # 计算系数
    coe = 255.0 / Diff
    rgbDiff = img - Shadow
    rgbDiff = np.maximum(rgbDiff, 0)
    img = rgbDiff * coe
    # 四舍五入到整数
    img = np.around(img, 0)
    # 变为int型
    img = img.astype(np.uint8)
    return img

##################################################################

# 計算兩張圖片的相似度（https://www.twblogs.net/a/5ee8804f38b2869e41a03399）

# 均值哈希算法
def aHash(img):
    # 縮放爲8*8
    img = cv2.resize(img, (8, 8))
    # 轉換爲灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # s爲像素和初值爲0，hash_str爲hash值初值爲''
    s = 0
    hash_str = ''
    # 遍歷累加求像素和
    for i in range(8):
        for j in range(8):
            s = s + gray[i, j]
    # 求平均灰度
    avg = s / 64
    # 灰度大於平均值爲1相反爲0生成圖片的hash值
    for i in range(8):
        for j in range(8):
            if gray[i, j] > avg:
                hash_str = hash_str + '1'
            else:
                hash_str = hash_str + '0'
    return hash_str
# 差值感知算法
def dHash(img):
    img = cv2.resize(img, (9, 8))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hash_str = ''
    # 每行前一個像素大於後一個像素爲1，相反爲0，生成哈希
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j + 1]:
                hash_str = hash_str + '1'
            else:
                hash_str = hash_str + '0'
    return hash_str
# 感知哈希算法(pHash)
def pHash(img):
    img = cv2.resize(img, (32, 32))  # , interpolation=cv2.INTER_CUBIC
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 將灰度圖轉爲浮點型，再進行dct變換
    dct = cv2.dct(np.float32(gray))
    # opencv實現的掩碼操作
    dct_roi = dct[0:8, 0:8]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash
# Hash值對比
def cmpHash(hash1, hash2):
    n = 0
    # hash長度不同則返回-1代表傳參出錯
    if len(hash1)!=len(hash2):
        return -1
    # 遍歷判斷
    for i in range(len(hash1)):
        # 不相等則n計數+1，n最終爲相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n

##################################################################

# interpolating between two images
import matplotlib.cm as cm
from scipy import interpolate
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage.interpolation import shift

def inter(images,t):
    # input: 
    # images: list of arrays/frames ordered according to motion
    # t: parameter ranging from 0 to 1 corresponding to first and last frame 
    # returns: interpolated image

    #direction of movement, assumed to be approx. linear 
    a=np.array(center_of_mass(images[0]))
    b=np.array(center_of_mass(images[-1]))

    #find index of two nearest frames
    arr=np.array([center_of_mass(images[i]) for i in range(len(images))])
    v=a+t*(b-a) #convert t into vector 
    idx1 = (np.linalg.norm((arr - v),axis=1)).argmin()
    arr[idx1]=np.array([0,0]) #this is sloppy, should be changed if relevant values are near [0,0]
    idx2 = (np.linalg.norm((arr - v),axis=1)).argmin()

    if idx1>idx2:
        b=np.array(center_of_mass(images[idx1])) #center of mass of nearest contour
        a=np.array(center_of_mass(images[idx2])) #center of mass of second nearest contour
        tstar=np.linalg.norm(v-a)/np.linalg.norm(b-a) #define parameter ranging from 0 to 1 for interpolation between two nearest frames
        im1_shift=shift(images[idx2],(b-a)*tstar) #shift frame 1
        im2_shift=shift(images[idx1],-(b-a)*(1-tstar)) #shift frame 2
        return im1_shift+im2_shift #return average

    if idx1<idx2:
        b=np.array(center_of_mass(images[idx2]))
        a=np.array(center_of_mass(images[idx1]))
        tstar=np.linalg.norm(v-a)/np.linalg.norm(b-a)
        im1_shift=shift(images[idx2],-(b-a)*(1-tstar))
        im2_shift=shift(images[idx1],(b-a)*(tstar))
        return im1_shift+im2_shift

In [ ]:
#superimpose original image and label
path = "20221005/001/001-MR/"
images_path = glob.glob(path + '*_0.png')
labels_path = glob.glob(path + '*_anno.png')
images = []
for i in range(len(images_path)):
    image = Image.open(images_path[i])
    label = Image.open(labels_path[i])
    image.paste(label,(0,0),mask=label)
    images.append(image)
    
#show superimposed images with pyplot
showImagesWithPyPlot(images)

In [ ]:
#draw lines with pillow and add images to list
path = "20221005/002/002-MR/"
images_path = glob.glob(path + '*_0.png')
labels_path = glob.glob(path + '*_anno.png')
jsonFiles = glob.glob(path + "*.json")

labelImages = []
for i in jsonFiles:
    labelImages.append(getLabeledImageWithJson(i))

#show filledLabels with pyplot
print("Quantity: " + str(len(labelImages)))
showImagesWithPyPlot(labelImages)
    
#superimpose filledLabelImages to original mri images and show
images=[]
for i in range(len(labelImages)):
    image = Image.open(images_path[i])
    labelImage = labelImages[i]
#     alpha = Image.new("L", labelImage.size, 100)
#     labelImage.putalpha(alpha)
    image.paste(labelImage,(0,0),mask=labelImage)
    images.append(image)
showImagesWithPyPlot(images)

In [ ]:
# categorize dicom files with patientID, studyID and seriesID using dicomdir
# dict: {patientID->studyID->seriesID->{dcmPath,label}}
# find images file name by SeriesNumber to be drawn
from pydicom.fileset import FileSet
dicomdir = dcmread("002/dicomdir")
fs = FileSet(dicomdir)
# print(fs)

dict = {}
patientIDs = fs.find_values("PatientID")
for patientID in patientIDs:
    instances = fs.find(PatientID=patientID)
    for ds in instances:
        patientID = ds.PatientID
        studyID = ds.StudyID
        seriesID = ds.SeriesNumber
        fileName = ds.ReferencedFileID
        data = {'dcmPath':fileName,'labelPath':'NA'}
        if patientID in dict:
            if studyID in dict[patientID]:
                if seriesID in dict[patientID][studyID]:
                    dict[patientID][studyID][seriesID].append(data)
                else:
                    dict[patientID][studyID][seriesID] = [data]
            else:
                dict[patientID][studyID] = {seriesID:[data]}
        else:
            dict[patientID] = {studyID:{seriesID:[data]}}

#draw images
drawCategorizedImages('002/', dict, fontsize=15, fontfamily='DejaVu Sans')

In [162]:
# 按照series對MRI與PET做資料夾分類，并且畫出MRI對應的標記圖像
patientFolder = '005'
mri_dict = getCategorizedDictByFolder('./20221005/'+ patientFolder+ '/'+ patientFolder+ '-MR/')
pet_dict = getCategorizedDictByFolder('./20221005/'+ patientFolder+ '/'+ patientFolder+ '-PT/')

for patient in mri_dict:
    for study in mri_dict[patient]:
        for series in mri_dict[patient][study]:
            seriesDescripiton = dcmread(mri_dict[patient][study][series][0]['dcmPath']).SeriesDescription
            seriesDescripiton = seriesDescripiton.replace('/',' ')
            path = 'categorized/'+ patientFolder+ '/MRI/'+ series + ' (' + seriesDescripiton + ')'
            labelPath = 'categorized/'+ patientFolder+ '/MRI/'+ series + ' (' + seriesDescripiton + ')' + '/label/'
            if not os.path.exists(path):
                os.makedirs(path)
            if not os.path.exists(labelPath):
                os.makedirs(labelPath)
            for c,v in enumerate(mri_dict[patient][study][series]):
                shutil.copy(v['dcmPath'],path+ '/'+ str(c+1).zfill(3)+ '.dcm')
                width = dcmread(v['dcmPath']).pixel_array.shape[0]
                height = dcmread(v['dcmPath']).pixel_array.shape[1]
                labelImg = Image.new('RGBA', (width, height))
                if(v['labelPath']!='NA'):                    
                    labelImg = getLabeledImageWithJson(v['labelPath'])
                labelImg.save(labelPath + str(c+1).zfill(3) + '.png')                  
                
for patient in pet_dict:
    for study in pet_dict[patient]:
        for series in pet_dict[patient][study]:
            seriesDescripiton = dcmread(pet_dict[patient][study][series][0]['dcmPath']).SeriesDescription
            seriesDescripiton = seriesDescripiton.replace('/',' ')
            path = 'categorized/'+ patientFolder+ '/PET/'+ series + ' (' + seriesDescripiton + ')'
            if not os.path.exists(path):
                os.makedirs(path)
            for c,v in enumerate(pet_dict[patient][study][series]):
                shutil.copy(v['dcmPath'],path+ '/'+ str(c+1).zfill(3)+ '.dcm')

In [191]:
# convert MRI dicom to png 
import dicom2jpg
mriDicomFolder = 'categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/'
dicom2jpg.dicom2png(mriDicomFolder, target_root=mriDicomFolder)

True

In [173]:
# convert PET dicom to png 
import dicom2jpg
petDicomFolder = 'categorized/003/MRI/701 (T2W_FLAIR_FS+CM_MultiHance)/'
dicom2jpg.dicom2png(petDicomFolder, target_root=mriDicomFolder)

True

In [192]:
# rename files (因轉換後檔案名稱在排序上有問題，需要重新命名)
renameFolder = 'categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/png/'
files = glob.glob(renameFolder + "*.png")
files.sort(key=lambda x: len(x), reverse=False)
for c,v in enumerate(files):
    os.rename(v, renameFolder + str(c+1).zfill(3) + '.png')

In [ ]:
# try to remove hand from PET image
ds= io.imread('C:/Users/Justin/Desktop/MRI/pet/gray/003/040.png')
cols = ds.shape[0]
rows = ds.shape[1]
p = 0
for i in range(cols):
    for j in range(rows//2,rows):
        if(ds[j][i]==0):
            p = j
            break
    for k in range(p,rows):
        ds[k][i]=0

for i in range(cols):
    for j in range(rows//2,0,-1):
        if(ds[j][i]==0):
            p = j
            break
    for k in range(p,0,-1):
        ds[k][i]=0

plt.imshow(ds,cmap='gray')

In [210]:
# Find the most similar image
from image_similarity_measures.quality_metrics import rmse,ssim,fsim,sre
flairImg = cv2.imread('categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/png/025.png')
flairImg = cv2.resize(flairImg, (512, 512))
petPaths = glob.glob('pet/005/*.png')

similarityDict = []
print('No,SSIM,FSIM,RMSE,SRE,PHash')
for i in range(28,40):
    img2 = cv2.imread(petPaths[i])
    _ssim = ssim(flairImg,img2)
    _fsim = fsim(flairImg,img2)
    _rmse = rmse(flairImg,img2)
    _sre = sre(flairImg,img2)
    _pHash = cmpHash(pHash(flairImg),pHash(img2))
    print(str(i+1),end=',')
    print(str(_ssim),end=',')
    print(str(_fsim),end=',')
    print(str(_rmse),end=',')
    print(str(_sre),end=',')
    print(str(_pHash))
    similarityDict.append({'No':i+1,'SSIM':_ssim,'FSIM':_fsim,'RMSE':_rmse,'SRE':_sre,'PHash':_pHash})
similarityDict.sort(key=lambda x:x['SSIM'],reverse=True)
print(similarityDict[0]['No'])


No,SSIM,FSIM,RMSE,SRE,PHash
29,0.7206196541831553,0.1944834434287103,0.013994463,40.34267372969542,29
30,0.7208609486852676,0.19492988627191768,0.013983566,40.34605629354008,28
31,0.7210451418793289,0.19559817262128257,0.013973847,40.349075813861084,30
32,0.7211223076946642,0.19515231695687577,0.013969961,40.35028415214168,33
33,0.7211540476767194,0.1952464834188993,0.013967536,40.3510377602109,32
34,0.721126607256032,0.19539490329297862,0.013968865,40.35062473667261,35
35,0.7210828949490109,0.1950923867261742,0.01397594,40.34842588835243,34
36,0.7209689601687228,0.19578320856496592,0.013991938,40.34345753506243,32
37,0.7207689210106519,0.19636311550396168,0.014015126,40.33626634707413,32
38,0.7204185509673473,0.19750608207828776,0.014050321,40.325374424748794,35
39,0.7200145362817035,0.1980598499667093,0.014085743,40.31443938088296,34
40,0.7196364217408507,0.19897353043318913,0.014096525,40.31111664442644,33
33


In [189]:
# 挑選PET内插後與MRI位置對應的圖像
# 13*(n-1)+n
mriSlicesQuantity = 49 #mri切片總數量
mriSliceNum = 25    #mri對應號碼
petSliceNum = 34    #pet對應號碼
start = 13*(petSliceNum-1)+petSliceNum
src_path = 'C:/Users/Justin/Desktop/high_order_slices_interpolation_for-medical_images/interpolated/002/'
dst_path = 'pet/002/corresponded/'
if not os.path.exists(dst_path):
    os.makedirs(dst_path)
n = 0
m = 0
print(start)
for i in range(start,0,-1):
    if(n%30==0):
        m+=1
        if(m>mriSliceNum):
            break
        shutil.copy(src_path+str(i)+'.png',dst_path+str(i)+'.png')
        n=0
    n+=2
for i in range(start,967):
    if(n%30==0):
        if(m>mriSlicesQuantity+1):
            break        
        m+=1
        shutil.copy(src_path+str(i)+'.png',dst_path+str(i)+'.png')
        n=0
    n+=2

407


In [204]:
# 計算MRI與PET之間的voxel物理大小 與 兩者之間的縮放比例
mri = dcmread('categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/001.dcm')
pet = dcmread('categorized/004/PET/3 (WB Standard)/001.dcm')
mriPixelSpacing = mri.PixelSpacing
petPixelSpacing = pet.PixelSpacing
print(mri.pixel_array.shape)
print('MRI: ' + str(mriPixelSpacing))
print(pet.pixel_array.shape)
print('PET: ' + str(petPixelSpacing))
print('Ratio: ' + str(petPixelSpacing[0]/mriPixelSpacing[0]))

(512, 512)
MRI: [0.44921875, 0.44921875]
(512, 512)
PET: [1.367188, 1.367188]
Ratio: 3.0434793739130437


In [ ]:
# rotate and rescale pet images
from skimage.transform import rotate,rescale,resize
mirFolder = 'categorized/005/MRI/6 (t2_tse_dark-fluid_fs_tra+C)/'
petFolder = 'categorized/005/PET/3 (WB Standard)/'
correspondedPETPaths = glob.glob('pet/005/corresponded/*.png')
saveFolder = 'pet/005/corresponded/shaped/'
if not os.path.exists(saveFolder):
    os.makedirs(saveFolder)

mri = dcmread(mirFolder + '001.dcm')
mri2 = dcmread(mirFolder + '002.dcm')
pet = dcmread(petFolder + '001.dcm')
mriPixelSpacing = mri.PixelSpacing
petPixelSpacing = pet.PixelSpacing
imgSize = mri.pixel_array.shape[0]
shiftX = 36
shiftY = 100
rotateVal = -6
pixelRescaleVal = mri.pixel_array.shape[0] / pet.pixel_array.shape[0]   #像素比例
mmRescaleVal = (petPixelSpacing[0]/mriPixelSpacing[0])/pixelRescaleVal   #物理比例
deltaXVal = (mri2.ImagePositionPatient[0] - mri.ImagePositionPatient[0]) / mri.PixelSpacing[0]  #mm轉換成像素單位
deltaYVal = (mri2.ImagePositionPatient[1] - mri.ImagePositionPatient[1]) / mri.PixelSpacing[0]  #mm轉換成像素單位
deltaX = 0
deltaY = 0
print(pixelRescaleVal)
print(mmRescaleVal)
for i in range(28,0,-1):
    img = io.imread(correspondedPETPaths[i-1])
    img = rescale(img,pixelRescaleVal)
    img = rescale(img,mmRescaleVal)    
    img = rotate(img,rotateVal)
    end = int((img.shape[0]/2) + (imgSize/2))
    start = end - imgSize
    img = img[int(start-shiftY+deltaY):int(end-shiftY+deltaY),int(start-shiftX+deltaX):int(end-shiftX+deltaX)]
    io.imsave(saveFolder+str(i).zfill(3)+'.png', img)
#     deltaX += deltaXVal
#     deltaY += deltaYVal

# deltaX = deltaXVal
# deltaY = deltaYVal
for i in range(28,len(correspondedPETPaths)):
    img = io.imread(correspondedPETPaths[i])
    img = rescale(img,pixelRescaleVal)
    img = rescale(img,mmRescaleVal)    
    img = rotate(img,rotateVal)
    end = int((img.shape[0]/2) + (imgSize/2))
    start = end - imgSize
    img = img[int(start-shiftY-deltaY):int(end-shiftY-deltaY),int(start-shiftX-deltaX):int(end-shiftX-deltaX)]
    io.imsave(saveFolder+str(i+1).zfill(3)+'.png', img)
    # deltaX += deltaXVal
    # deltaY += deltaYVal

In [ ]:
#draw comparison of FLAIR and shaped PET
flairPath = glob.glob('categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/png/*.png')
petPath = glob.glob('pet/004/corresponded/shaped/*.png')
cols = 2
rows = 47
fig = plt.figure(figsize=(4, 100))
for n_slice in range(47):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)    
    img1 = io.imread(flairPath[n_slice])
    plt.imshow(img1,cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    img2 = io.imread(petPath[n_slice])
    plt.imshow(img2,cmap='gray')
    plt.axis('off')

In [122]:
# print deltaX, deltaY and deltaZ of slices of dicom
dcmFolder = glob.glob('categorized/004/MRI/601 (T2W_FLAIR_FS+C(Gadovist))/*dcm')
dss = [dcmread(i) for i in dcmFolder]
preX = 0
preY = 0
preZ = 0
for i in dss:
    # print('SliceLocation: ' + str(i.ImagePositionPatient))
    pos = i.ImagePositionPatient
    curX = pos[0]
    curY = pos[1]
    curZ = pos[2]
    print(str(curX - preX)+', '+str(curY - preY)+ ', ' + str(curZ - preZ))
    preX = curX
    preY = curY
    preZ = curZ

124.93737152453, -92.427999222898, 118.323310950894
0.09947013855000364, -0.551567077637003, -2.94718170166
0.09947013855000364, -0.5515670776360082, -2.94718170166
0.09947037695999938, -0.5515661239629992, -2.94718170166
0.09947013855000364, -0.5515670776369888, -2.94718170166
0.09947013854998943, -0.5515670776360082, -2.9471817016609947
0.09947037697000383, -0.5515661239629992, -2.94718170166
0.09947013855000364, -0.551567077635994, -2.9471817016595026
0.09947037697000383, -0.5515666008000011, -2.9471817016600994
0.09946990012998924, -0.5515666008000011, -2.947181701660199
0.09947061539000401, -0.5515670776360082, -2.9471817016600994
0.09947013855000364, -0.5515666008000011, -2.947185516357507
0.09947013855000364, -0.5515666007989921, -2.9471817016600994
0.09947013854998943, -0.5515666008000011, -2.947181701660199
0.09947013855000364, -0.5515670776290023, -2.9471817016600994
0.09947061537999957, -0.5515666008000011, -2.947181701660199
0.09947013855000364, -0.5515666008000011, -2.9471

In [8]:
#Prepare dataset for input->(128,128,48,3)
SIZE = 240
DEPTH = 48
CHANNEL = 3
patientID = list(dict)[0]
studyID = list(dict[patientID])[0]
t1 = dict[patientID][studyID][getSeriesIDByName(dict, 't1w_ff')]
t2 = dict[patientID][studyID][getSeriesIDByName(dict, 't2w_fs')]
flair = dict[patientID][studyID][getSeriesIDByName(dict, 'flair')]

t1_ds = [dcmread(i['dcmPath']) for i in t1]
t2_ds = [dcmread(i['dcmPath']) for i in t2]
flair_ds = [dcmread(i['dcmPath']) for i in flair]

t1s = np.zeros((SIZE,SIZE,DEPTH))
t2s = np.zeros((SIZE,SIZE,DEPTH))
flairs = np.zeros((SIZE,SIZE,DEPTH))
t1s_seg = np.zeros((SIZE,SIZE,DEPTH))
t2s_seg = np.zeros((SIZE,SIZE,DEPTH))
flairs_seg = np.zeros((SIZE,SIZE,DEPTH))

for c, i in enumerate(t1_ds):
    t1s[:,:,c] = cv2.resize(i.pixel_array, (SIZE,SIZE))
for c, i in enumerate(t2_ds):
    t2s[:,:,c] = cv2.resize(i.pixel_array, (SIZE,SIZE))
for c, i in enumerate(flair_ds):
    flairs[:,:,c] = cv2.resize(i.pixel_array, (SIZE,SIZE))
for c, i in enumerate(t1):
    if(i['labelPath'] != 'NA'):
        label = getLabeledImageWithJson(i['labelPath'])
        label = label.convert('L')
        t1s_seg[:,:,c] = cv2.resize(np.array(label), (SIZE,SIZE))
for c, i in enumerate(t2):
    if(i['labelPath'] != 'NA'):
        label = getLabeledImageWithJson(i['labelPath'])
        label = label.convert('L')
        t2s_seg[:,:,c] = cv2.resize(np.array(label), (SIZE,SIZE))
for c, i in enumerate(flair):
    if(i['labelPath'] != 'NA'):
        label = getLabeledImageWithJson(i['labelPath'])
        label = label.convert('L')
        flairs_seg[:,:,c] = cv2.resize(np.array(label), (SIZE,SIZE))
        
t1s_seg[t1s_seg!=0] = 1
t2s_seg[t2s_seg!=0] = 1
flairs_seg[flairs_seg!=0] = 1

In [ ]:
fig = plt.figure(figsize=(4, 100))
for n_slice in range(48):
    ax=fig.add_subplot(48,2,(n_slice*2)+1)
    ax.title.set_text('t2')
    plt.imshow(flairs[:,:,n_slice], cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(48,2,(n_slice*2)+2)
    ax.title.set_text('Label '+str(n_slice+1))
    plt.imshow(flairs_seg[:,:,n_slice])
    plt.axis('off')

In [13]:
#save combined channels to npy files
data_3channels = np.stack([flairs,t1s,t2s], axis=3)
print(data_3channels.shape)
print(flairs_seg.shape)
if(not os.path.exists('input_data_3channels')):
    os.makedirs('input_data_3channels')
np.save('input_data_3channels/'+path.split('/')[1]+'.npy', data_3channels)
np.save('input_data_3channels/'+path.split('/')[1]+'_seg.npy', flairs_seg)
nib.save(nib.Nifti1Image(t1s, affine=np.eye(4)), 'input_data_3channels/'+path.split('/')[1]+'_t1.nii')
nib.save(nib.Nifti1Image(t2s, affine=np.eye(4)), 'input_data_3channels/'+path.split('/')[1]+'_t2.nii')
nib.save(nib.Nifti1Image(flairs, affine=np.eye(4)), 'input_data_3channels/'+path.split('/')[1]+'_flair.nii')
nib.save(nib.Nifti1Image(flairs_seg, affine=np.eye(4)), 'input_data_3channels/'+path.split('/')[1]+'_seg.nii')

(240, 240, 48, 3)
(240, 240, 48)


In [ ]:
### 嘗試用標記檔配對dcm檔
path = "20221005/002/002-MR"
files = glob.glob(path + "/*.dcm")
pngs = glob.glob(path + "/*0.png")
labes = glob.glob(path + "/*anno.png")
cols = 3
rows = len(pngs)
fig = plt.figure(figsize=(rows/6,rows*2))
for i in range(len(pngs)):
    sopID = str(pngs[i].split(path)[1][1:-6]) #去除資料夾字段及後綴，保留UID
    for j in files:
        f = dcmread(j)
        if(str(f.SOPInstanceUID) == sopID):
            ax1 = fig.add_subplot(rows,cols,(i*cols)+1)
            ax1.title.set_text('dcm/series: ' + str(f.SeriesNumber))
            plt.imshow(f.pixel_array)
            plt.axis('off')
            ax2 = fig.add_subplot(rows,cols,(i*cols)+2)
            ax2.title.set_text('png')
            plt.imshow(Image.open(pngs[i]))
            plt.axis('off')
            ax3 = fig.add_subplot(rows,cols,(i*cols)+3)
            ax3.title.set_text('label')
            plt.imshow(Image.open(labes[i]))
            plt.axis('off')

In [ ]:
#plot interpolated images:
# flair = nib.load('./input_data_3channels/003_flair.nii').get_fdata()
# flair = interpolate_output
# print(flair.shape)
# slice_number = flair.shape[2]
# interpolate_output = np.zeros((240,240,(slice_number-1)*2 + 1))
# for i in range(slice_number-1):
#     interpolate_output[:,:,(i*2)] = flair[:,:,i]
#     out = inter([flair[:,:,i],flair[:,:,i+1]], 0)
#     interpolate_output[:,:,(i*2)+1] = out
#     interpolate_output[:,:,(i*2)+2] = flair[:,:,i+1]
# print(interpolate_output.shape)

cols = 3
rows = 47

fig = plt.figure(figsize=(6,100))
for n_slice in range(rows-1):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    ax.title.set_text('interpolated')
    plt.imshow(interpolate_output[:,:,(n_slice*4)+1],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    ax.title.set_text('interpolated')
    plt.imshow(interpolate_output[:,:,(n_slice*4)+2],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    ax.title.set_text('interpolated')
    plt.imshow(interpolate_output[:,:,(n_slice*4)+3],cmap='gray')
    plt.axis('off')

In [ ]:
def save_nii_to_png(niifile):    
    img = nib.load(niifile)
    fdata = img.get_fdata()
    (x,y,z) = fdata.shape
    for i in range(z):
        slice_pic = fdata[:,:,i]
        plt.imsave('nii_to_png/img' + str(i+1) + '.png', slice_pic, cmap='gray')

TRAIN_DATASET_PATH = ''
test_image = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_001_seg.nii')
print(test_image)
test_image = test_image.get_fdata()
plt.figure(figsize=(20,20))

for i in range(test_image.shape[2]):
    plt.subplot(11,15,i+1)
    plt.imshow(test_image[:,:,i])
    plt.axis('off')

if(not os.path.exists('nii_to_png')):
    os.mkdir('nii_to_png')
save_nii_to_png(TRAIN_DATASET_PATH + 'BraTS20_Training_001_seg.nii')

In [ ]:
#load PET nii file
path = './FDG-PET-CT-Lesions/PETCT_0b98dbe00d/'
ct = nib.load(path + 'CT.nii.gz').get_fdata()
ctres = nib.load(path + 'CTres.nii.gz').get_fdata()
suv = nib.load(path + 'SUV.nii.gz').get_fdata()
pet = nib.load(path + 'PET.nii.gz').get_fdata()
seg = nib.load(path + 'SEG.nii.gz').get_fdata()
print("CT: "+str(ct.shape))
print("CTres: "+str(ctres.shape))
print("SUV: "+str(suv.shape))
print("PET: "+str(pet.shape))
print("LABEL: "+str(seg.shape))
print(np.unique(seg))

In [ ]:
flair = nib.load('./input_data_3channels/003_flair.nii').get_fdata()
showNiftiImagesWithPyPlot(flair,'gray')

In [ ]:
flair_interpolated = nib.load('./input_data_3channels/003_flair_interpolated.nii').get_fdata()
showNiftiImagesWithPyPlot(flair_interpolated,'gray')

In [ ]:
#draw comparison of original and interpolated images
cols = 3
rows = 47

fig = plt.figure(figsize=(6,100))
for n_slice in range(rows-1):
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+1)
    # ax.title.set_text(str(n_slice*2))
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+1],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+2)
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+2],cmap='gray')
    plt.axis('off')
    ax=fig.add_subplot(rows,cols,(n_slice*cols)+3)
    # ax.title.set_text(str((n_slice*2)+2))
    ax.title.set_text('interpolated')
    plt.imshow(flair_interpolated[:,:,(n_slice*4)+3],cmap='gray')
    plt.axis('off')

In [ ]:
#calculate images similarity
from image_similarity_measures.quality_metrics import ssim

slices = 47
total_ssim = 0
for i in range(slices-1):
    a = (interpolate_output[:,:,(i*4)+1]/np.max(interpolate_output[:,:,(i*4)+1]))*255
    b = (flair_interpolated[:,:,(i*4)+1])*255
    total_ssim += ssim(a,b)
    a = (interpolate_output[:,:,(i*4)+2]/np.max(interpolate_output[:,:,(i*4)+2]))*255
    b = (flair_interpolated[:,:,(i*4)+2])*255
    total_ssim += ssim(a,b)
    a = (interpolate_output[:,:,(i*4)+3]/np.max(interpolate_output[:,:,(i*4)+3]))*255
    b = (flair_interpolated[:,:,(i*4)+3])*255
    total_ssim += ssim(a,b)
print(total_ssim)
print("total_similarity: "+  str(total_ssim/((slices-1)*3)))
# fig = plt.figure(figsize=(4,2))
# ax=fig.add_subplot(1,2,1)
# plt.imshow(a,cmap='gray')
# plt.axis('off')
# ax=fig.add_subplot(1,2,2)
# plt.imshow(b,cmap='gray')
# plt.axis('off')